## Saver

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [ ]:
x_data = datasets.load_boston().data[:, 12] # 주변 이웃 중 하위 계층의 비율
y_data = datasets.load_boston().target # 집의 가격 (in $1,000s)
df = pd.DataFrame([x_data, y_data]).transpose()
df.head()

In [ ]:
# Gradient descent의 적용을 받지 않고 학습 횟수(learning step)에 따라 단순히 증가됨
global_step = tf.Variable(0, trainable=False)

In [ ]:
w = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.random_normal([1]))

y_predicted = w * x_data + b 

In [ ]:
loss = tf.reduce_mean(tf.square(y_predicted - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss, global_step=global_step) # Gradient descent가 1회 적용될 때마다 global_step은 1씩 증가

In [ ]:
sess = tf.Session()

In [ ]:
# Saver 객체변수 생성
saver = tf.train.Saver(tf.global_variables()) # 정의한 Variable들을 save or restore의 대상으로 세팅해줌

In [ ]:
ckpt = tf.train.get_checkpoint_state('./saved_model')
print(ckpt) # 처음 실행할 때에는 None

In [ ]:
if ckpt is not None and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path) # 저장이 되어있는 session이 있을 경우 restore
    print(saver)
else:
    sess.run(tf.global_variables_initializer()) # 없을 경우 variable 초기화 진행

In [ ]:
for step in range(100):
    sess.run(train)
    print('Global_Step {}: w {} b {}'.format(sess.run(global_step), sess.run(w), sess.run(b)))
    print('loss {}'.format(sess.run(loss)))
    print()

w_out, b_out = sess.run([w, b]) 

In [ ]:
# 100번 학습 진행 후 Save & 결과 체크

ckpt_path = saver.save(sess, './saved_model/train_phase1.ckpt', global_step=global_step)
print('Current session is saved (as ckpt files) @ {}'.format(ckpt_path))
sess.close() 

# 저장이 완료된 것을 확인한 후, 상단의 "sess = tf.Session()" 셀부터 다시 실행하여 Global step이 증가되는 것을 확인해보세요.

In [ ]:
# 결과 확인
plt.figure(figsize = (10,10))
plt.plot(x_data, y_data, 'bo', label='Real data')
plt.plot(x_data, x_data * w_out + b_out, 'ro', label='Prediction')
plt.legend()
plt.show()